# TP7: Décomposition QR

La décomposition QR est une décomposition centrale en analyse numérique. Elle est principalement utilisée pour trouver une base de l'image d'une matrice, mais elle est aussi utilisée pour résoudre des systèmes linéaires, pour calculer les valeurs propres d'une matrice, etc.

Au programme de ce TP:
- Introduction à la décomposition QR
  - Fonctions de numpy et scipy
  - Propriétés de la décomposition QR
- Algorithmes de décomposition QR
  - Gram-Schmidt
  - Gram-Schmidt modifié
  - Householder
- Stabilité numérique des algorithmes
- Application à la résolution de systèmes linéaires
- Bonus: Algorithme de Givens

## Échauffement: Introduction à la décomposition QR

Commençons par nous familiariser avec la décomposition QR. 

La décomposition QR d'une matrice $A$ est une décomposition de la forme 
$$A = QR$$ 
où $Q$ est une matrice orthogonale et $R$ est une matrice triangulaire supérieure.

Les deux principaux packages de Python pour le calcul scientifique sont `numpy` et `scipy`. Chaque package contient des fonctions pour calculer la décomposition QR. 
- Pour numpy, la fonction est `numpy.linalg.qr`. La documentation est disponible [ici](https://docs.scipy.org/doc/numpy/reference/generated/numpy.linalg.qr.html).
- Pour scipy, la fonction est `scipy.linalg.qr`. La documentation est disponible [ici](https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.qr.html).

Comme vous pouvez le voir dans la documentation, `scipy` offre plus de possibilités que `numpy`. Nous l'utiliserons donc pour ce TP.

### Exercice 0.1

Après avoir importé numpy et scipy, générez une matrice aléatoire $A$ de taille $4 \times 3$ et calculez la décomposition QR de $A$ avec la fonction `scipy.linalg.qr`. Affichez les matrices $Q$ et $R$.

Comparez les résultats en ajoutant l'argument `mode='economic'` à la fonction `scipy.linalg.qr`. Quelle est la différence entre les deux décompositions?


In [ ]:
import numpy as np
import scipy as sp
# VOTRE CODE ICI


### Exercice 0.2

Vérifiez les propriétés suivantes de la décomposition QR:
- $A = QR$.
- $Q$ est orthogonale.
- $||A||_F = ||R||_F$.

In [ ]:
# VOTRE CODE ICI


## Exercice 1: Algorithmes de décomposition QR

Le but de cet exercice est d'implémenter soi-même les différents algorithmes de décomposition QR vus en cours.

### Exercice 1.1: Méthode de Gram-Schmidt

Implémentez une fonction `GramSchmidt` qui prend en argument une matrice $A$ et qui renvoie la matrice $Q$ telle que $$A = QR,$$ où $Q$ est une matrice orthogonale et $R$ est une matrice triangulaire supérieure, au moyen de la procédure d'orthogonalisation de Gram-Schmidt. 

Voir l'algorithme 6.6 du polycopié.

Comme lors de l'échauffement, testez votre code sur une matrice aléatoire de taille $4 \times 3$ en vérifiant les propriétés de $Q$ et $R$, et $||A - QR||_2$.


In [ ]:
def GramSchmidt(A):
    """
    Décomposition QR de A par la méthode de Gram-Schmidt classique (non stable).

    Entrées
    -------
    A : ndarray
        Matrice de taille m x n. (m >= n)

    Sorties
    -------
    Q : ndarray
        Matrice orthogonale de taille m x n.
    R : ndarray
        Matrice triangulaire supérieure de taille n x n.
    """
    m, n = A.shape
    Q = np.zeros((m,n))
    R = np.zeros((n,n))
    # VOTRE CODE ICI
    
    return Q, R


In [ ]:
# VOTRE TEST ICI




In [ ]:
# TEST AUTOMATIQUE
from numpy.testing import assert_allclose
from numpy.random import randn
from tqdm import tqdm

all_m = np.linspace(10, 100, 10, dtype=int)
all_n = np.linspace(10, 100, 10, dtype=int)

for m, n in tqdm(zip(all_m, all_n)):
    if m < n:
        continue
    A = randn(m, n)
    Q, R = GramSchmidt(A)
    # Check that A = QR
    assert_allclose(A, Q @ R, atol=1e-10, err_msg="A != QR")
    # Note: with Gram-Schmidt, Q is not necessarily orthogonal

print("Tests automatiques passés avec succès.")

### Exercice 1.2: Décomposition QR par la méthode de Gram-Schmidt modifiée

Similairement à la question précédente, écrire une fonction `GramSchmidtMod` qui utilise la méthode de Gram-Schmidt modifiée.

Voir l'Algorithme 6.7 du polycopié.

Comme précédemment, testez simplement votre code sur une matrice $4 \times 3$. 
Nous verrons dans l'exercice 1.4 que la méthode de Gram-Schmidt modifiée est plus stable numériquement que la méthode de Gram-Schmidt.

In [ ]:
def GramSchmidtMod(A):
    """
    Décomposition QR de A par la méthode de Gram-Schmidt modifiée (stable).

    Entrées
    -------
    A : ndarray
        Matrice de taille m x n. (m >= n)

    Sorties
    -------
    Q : ndarray
        Matrice orthogonale de taille m x n.
    R : ndarray
        Matrice triangulaire supérieure de taille n x n.
    """
    m, n = A.shape
    Q = np.zeros((m,n))
    R = np.zeros((n,n))
    # VOTRE CODE ICI
    
    return Q, R


In [ ]:
# VOTRE TEST ICI




In [ ]:
# TEST AUTOMATIQUE
from numpy.testing import assert_allclose
from numpy.random import randn
from tqdm import tqdm

all_m = np.linspace(10, 100, 10, dtype=int)
all_n = np.linspace(10, 100, 10, dtype=int)

for m, n in tqdm(zip(all_m, all_n)):
    if m < n:
        continue
    A = randn(m, n)
    Q, R = GramSchmidtMod(A)
    # Check that A = QR
    assert_allclose(A, Q @ R, atol=1e-10, err_msg="A != QR")
    # Note: with modified Gram-Schmidt, Q is not necessarily orthogonal

print("Tests automatiques passés avec succès.")

### Exercice 1.3: Décomposition QR par la méthode de Householder

Écrire une fonction `Householder` qui calcule la décomposition QR d'une matrice $A$ par la méthode de Householder.

Voir l'algorithme 6.11 du polycopié.

Dans cet algorithme, la mise à jour de $R$ est donnée par 
$$R_{k:m,k} = R_{k:m,k} - 2 v_k (v_k^T R_{k:m,k}).$$
Similairement, on peut obtenir (comment?) que la mise à jour de $Q$ est donnée par
$$Q_{k,k:m} = Q_{k,k:m} - 2 v_k (v_k^T Q_{k,k:m}).$$


In [ ]:
def Householder(A):
    """
    Décomposition QR de A par la méthode de Householder.

    Entrées
    -------
    A : ndarray
        Matrice de taille m x n. (m >= n)

    Sorties
    -------
    Q : ndarray
        Matrice orthogonale de taille m x n.
    R : ndarray
        Matrice triangulaire supérieure de taille n x n.
    """
    m, n = A.shape
    Q = np.eye(m)
    R = A.copy()
    # VOTRE CODE ICI
    
    return Q, R

In [ ]:
# VOTRE TEST ICI


In [ ]:
# TEST AUTOMATIQUE
from numpy.testing import assert_allclose
from numpy.random import randn
from tqdm import tqdm

all_m = np.linspace(10, 100, 10, dtype=int)
all_n = np.linspace(10, 100, 10, dtype=int)

for m, n in tqdm(zip(all_m, all_n)):
    if m < n:
        continue
    A = randn(m, n)
    Q, R = Householder(A)
    # Check that A = QR
    assert_allclose(A, Q @ R, atol=1e-10, err_msg="A != QR")
    # Check that Q is orthogonal
    assert_allclose(Q.T @ Q, np.eye(Q.shape[1]), atol=1e-10, err_msg="Q is not orthogonal")

print("Tests automatiques passés avec succès.")


## Exercice 2 : Stabilité numérique

Dans cet exercice, nous allons comparer la stabilité numérique des différents algorithmes de décomposition QR. Plus particulièrement, nous allons étudier l'orthogonalité des matrices $Q$ obtenues. Si ces matrices ne sont pas orthogonales, c'est suffisant pour montrer que la méthode correspondante n'est pas numériquement stable.

Pour cela, nous allons utiliser des matrices particulières. Pour $n=5$, la matrice de Frank $F_5$ est définie par $$F_5 = \begin{pmatrix} 1 & 1 & 1 & 1 & 1 \\ 1 & 2 & 2 & 2 & 2 \\ 0 & 2 & 3 & 3 & 3 \\ 0 & 0 & 3 & 4 & 4 \\ 0 & 0 & 0 & 4 & 5 \end{pmatrix}.$$

Le code ci-dessous permet de générer une matrice de Frank de taille $n \times n$.

Fixons $n=50$. Pour comparer les méthodes, considérons les matrices $$A_{50} = F_{50} + \alpha I_{50}, \quad \text{ où } \alpha \in [1, 10].$$


In [ ]:
def generate_Frank(n):
    """
    Génère la matrice de Frank de taille n x n.
    """
    F = np.zeros((n,n))
    for i in np.arange(n):
        for j in np.arange(n):
            if j < i-1:
                F[i,j] = 0
            elif j == i-1:
                F[i,j] = i
            else:
                F[i,j] = i + 1
    return F

# Matrice de Frank de taille 5 x 5
F = generate_Frank(5)
print("F = \n", F)

# Paramètres
n = 50
list_alpha = np.linspace(1, 10, 100)
Fn = generate_Frank(n)
In = np.eye(n)

# Boucle sur alpha
cond_A = np.zeros_like(list_alpha)
for i, alpha in enumerate(list_alpha):
    A = Fn + alpha * In
    cond_A[i] = np.linalg.cond(A)

# Plot du conditionnement en fonction de alpha
import matplotlib.pyplot as plt
plt.figure()
plt.semilogy(list_alpha, cond_A)
plt.grid()
plt.xlabel(r"$\alpha$")
plt.xlim(list_alpha[0], list_alpha[-1])
plt.ylabel(r"Conditionnement de $A$")
plt.show()

Comme on peut le voir sur le graphique ci-dessus, la conditionnement de la matrice $A_{50}$ est très élevé. Par conséquent, si l'on utilise une méthode de décomposition QR non stable, les erreurs d'arrondi peuvent se propager et conduire à des résultats faussés. 

### Exercice 2.1 : Stabilité numérique

Pour plusieurs valeurs $\alpha \in [1, 10]$:
- Calculez la décomposition QR de $A_{50}$ avec les méthodes implémentées dans les exercices précédents et la fonction `scipy.linalg.qr`.
- Pour chaque méthode, tracez $||Q^T Q - I_{50}||_1$ en fonction de $\alpha$. 
- Commentez les résultats.

Indication: Utilisez une échelle logarithmique pour l'axe des ordonnées avec `plt.semilogy`.

In [ ]:
# Paramètres
n = 50
nb_alpha = 100
list_alpha = np.linspace(1, 10, nb_alpha)
Fn = generate_Frank(n)
In = np.eye(n)

# Boucle sur alpha
err_ref = np.zeros_like(list_alpha)
err_gm = np.zeros_like(list_alpha)
err_gmm = np.zeros_like(list_alpha)
err_h = np.zeros_like(list_alpha)
for i, alpha in enumerate(list_alpha):
    A = Fn + alpha * In

    # Reference
    Q_ref, R_ref = sp.linalg.qr(A)
    err_ref[i] = np.linalg.norm(In - Q_ref.T @ Q_ref, ord=1)

    # VOTRE CODE ICI

# Plot
from matplotlib import pyplot as plt
fig = plt.figure(figsize=(12, 6))
plt.semilogy(list_alpha, err_ref, label="Référence (scipy)")
plt.semilogy(list_alpha, err_gm, label="Gram-Schmidt")
plt.semilogy(list_alpha, err_gmm, label="Gram-Schmidt modifié")
plt.semilogy(list_alpha, err_h, label="Householder")
plt.grid()
plt.xlabel(r"$\alpha$")
plt.xlim(list_alpha[0], list_alpha[-1])
plt.ylabel(r"$||I - Q^T Q||_1$")
plt.legend()
plt.show()


*Votre réponse ici*




## Exercice 3: Application à la résolution de systèmes linéaires

La décomposition QR peut être utilisée pour résoudre des systèmes linéaires. En effet,
$$A x = b \iff QR x = b \iff R x = Q^T b.$$
De plus, comme $R$ est triangulaire supérieure, on peut résoudre le système $Rx = y$ facilement (voir TP5). On pourra utiliser la fonction `scipy.linalg.solve_triangular` pour cela.

### Exercice 3.1

Écrire une fonction `QRsolve` qui prend en argument une matrice $A$ et un vecteur $b$ et qui renvoie la solution du système $Ax = b$. Utiliser l'argument `mode` pour choisir la méthode de décomposition QR à utiliser.


In [ ]:
def QRsolve(A, b, mode):
    """
    Résolution du système linéaire Ax = b par la méthode QR.
    """
    # VOTRE CODE ICI



### Exercice 3.2

Testez votre méthode avec les trois algorithmes sur le système linéaire suivant:

$$A = \begin{pmatrix} 2 & 1 & 3 \\ 2 & 6 & 8 \\ 6 & 8 & 18 \end{pmatrix}, \quad b = \begin{pmatrix} 2 \\ 3 \\ 1 \end{pmatrix}.$$

La solution exacte est $x = \begin{pmatrix} \frac{12}{5} & \frac{17}{10} & -\frac{3}{2} \end{pmatrix}^T$.

In [ ]:
# Données
A = np.array([[2, 1, 3], [2, 6, 8], [6, 8, 18]], dtype=float)
b = np.array([2, 3, 1], dtype=float)
x_ref = np.array([12/5, 17/10, -3/2], dtype=float)

# VOTRE CODE ICI



### Exercice 3.3

Pour $n=50$, considérons le système linéaire $$A_{50} x = b_{50},$$ où $A_{50}$ est la matrice de Frank de taille $50 \times 50$ et $b_{50}$ est un vecteur aléatoire de taille $50$.

- Que fait le code ci-dessous?
- Éditez le code pour comparer les trois méthodes de décomposition QR implémentées pour résoudre les systèmes linéaires.
- Commentez les résultats.

In [ ]:
# Paramètres
n = 50
np.random.seed(0)
b = np.random.randn(n)
list_alpha = np.linspace(3, 20, 100)

# Initialisation
cond_A = np.zeros_like(list_alpha)
forward_err_gm = np.zeros_like(list_alpha)
forward_err_gmm = np.zeros_like(list_alpha)
forward_err_h = np.zeros_like(list_alpha)
forward_err_scipy = np.zeros_like(list_alpha)

# Boucle sur alpha
for i, alpha in enumerate(list_alpha):
    A = Fn + alpha * In
    cond_A[i] = np.linalg.cond(A)

    # Reference
    x_ref = np.linalg.solve(A, b)

    # Gram-Schmidt
    x_gm = # VOTRE CODE ICI
    forward_err_gm[i] = np.linalg.norm(x_ref - x_gm, 2) / np.linalg.norm(x_ref, 2)

    # Gram-Schmidt modifié
    x_gmm = # VOTRE CODE ICI
    forward_err_gmm[i] = np.linalg.norm(x_ref - x_gmm, 2) / np.linalg.norm(x_ref, 2)

    # Householder
    x_h = # VOTRE CODE ICI
    forward_err_h[i] = np.linalg.norm(x_ref - x_h, 2) / np.linalg.norm(x_ref, 2)

    # Scipy
    x_scipy = # VOTRE CODE ICI
    forward_err_scipy[i] = np.linalg.norm(x_ref - x_scipy, 2) / np.linalg.norm(x_ref, 2)


# Plot
from matplotlib import pyplot as plt
fig = plt.figure(figsize=(12, 6))

plt.subplot(121)
plt.semilogy(list_alpha, cond_A, label="Conditionnement")
plt.grid()
plt.xlabel(r"$\alpha$")
plt.xlim(list_alpha[0], list_alpha[-1])
plt.ylabel(r"$\kappa(A)$")
plt.legend()

plt.subplot(122)
plt.semilogy(list_alpha, forward_err_gm, label="Gram-Schmidt")
plt.semilogy(list_alpha, forward_err_gmm, label="Gram-Schmidt modifié")
plt.semilogy(list_alpha, forward_err_h, label="Householder")
plt.semilogy(list_alpha, forward_err_scipy, label="Scipy")
plt.grid()
plt.xlabel(r"$\alpha$")
plt.xlim(list_alpha[0], list_alpha[-1])
plt.ylabel("Forward error")
plt.legend()


plt.show()



*Votre réponse ici*


## Exercice 4 (Bonus): Algorithme de Givens

Dans la série 15, un exercice porte sur l'algorithme de Givens. Cet algorithme permet de calculer la décomposition QR d'une matrice $A$ en effectuant des rotations de Givens sur les colonnes de $A$.

Dans certains cas, comme pour les matrices de Frank, l'algorithme de Givens est plus efficace que la méthode de Householder.

### Exercice 4.1

Comme dans la série 15, écrire une fonction `Givens_rotation` qui prend en argument deux scalaires $a$ et $b$ et qui renvoie la matrice de rotation de Givens correspondante.

In [ ]:
def Givens_rotation(a, b):
    """
    Calcul de la matrice de rotation de Givens.
    """
    # VOTRE CODE ICI
    

In [ ]:
# TEST AUTOMATIQUE
from numpy.testing import assert_allclose
# Cas b = 0
c, s = Givens_rotation(1, 0)
assert_allclose(c, 1, err_msg="Problème avec c lorsque b = 0")
assert_allclose(s, 0, err_msg="Problème avec s lorsque b = 0")
# Cas b > a
c, s = Givens_rotation(1, 2)
assert_allclose(s, 1 / np.sqrt(5/4), err_msg="Problème avec s lorsque b > a")
assert_allclose(c, - 1/2 * 1 / np.sqrt(5/4), err_msg="Problème avec c lorsque b > a")
# Cas b < a
c, s = Givens_rotation(2, 1)
assert_allclose(c, 1 / np.sqrt(5/4), err_msg="Problème avec c lorsque b < a")
assert_allclose(s, - 1/2 * 1 / np.sqrt(5/4), err_msg="Problème avec s lorsque b < a")

print("Tests passés avec succès !")

### Exercice 4.2

Écrire une fonction `Givens` qui calcule la décomposition QR d'une matrice $A$ par l'algorithme de Givens.

In [ ]:
def Givens(A):
    """
    Décomposition QR par la méthode de Givens.
    """
    m, n = A.shape
    Q = np.eye(m)
    R = A.copy()
    # VOTRE CODE ICI
    
    return Q, R

In [ ]:
# TEST AUTOMATIQUE
from numpy.testing import assert_allclose
from numpy.random import randn
from tqdm import tqdm

np.random.seed(0)
all_m, all_n = 10 * np.random.randint(1, 10, size=(2, 100))

for m, n in tqdm(zip(all_m, all_n)):
    A = randn(m, n)
    Q, R = Givens(A)
    # Check that A = QR
    assert_allclose(A, Q @ R, atol=1e-10, err_msg="A != QR")
    # Check that Q is orthogonal
    assert_allclose(Q.T @ Q, np.eye(Q.shape[1]), atol=1e-10, err_msg="Q is not orthogonal")

print("Tests automatiques passés avec succès.")

### Exercice 4.3

Considérons la matrice $A_{50}$ de l'exercice 3, avec $\alpha = 1$.

- Comparer la stabilité numérique de la méthode de Householder et de l'algorithme de Givens en calculant $||Q^T Q - I_{50}||_1$.
- Comparer le temps de calcul de chaque méthode avec la commande `%timeit`.

**Remarque:** Le temps de calcul dépend de votre implémentation. Pour optimiser votre méthode, vous pouvez économiser des calculs en détectant les cas où $b = 0$.

In [ ]:
n = 50
Fn = generate_Frank(n)
In = np.eye(n)
A = Fn + In

# Stabilité numérique
# VOTRE CODE ICI




In [ ]:
# Temps de calcul
print('Scipy:')
%timeit sp.linalg.qr(A)
# VOTRE CODE ICI